In [1]:
import pandas as pd 
import os 
import sys

sys.path.insert(0, os.getcwd() + '/code')
from viz import animate_play
from helper_functions import compute_orientation, face2face

In [2]:
df_game = pd.read_csv("data/games.csv")
df_tracking = pd.read_csv("data/week1.csv")
df_play = pd.read_csv("data/plays.csv")
df_players = pd.read_csv("data/players.csv")
df_pffScoutingData = pd.read_csv("data/pffScoutingData.csv")

In [3]:
df_tracking = compute_orientation(df_tracking)
print(df_tracking.head())

       gameId  playId    nflId  frameId                     time  \
0  2021090900      97  25511.0        1  2021-09-10T00:26:31.100   
1  2021090900      97  25511.0        2  2021-09-10T00:26:31.200   
2  2021090900      97  25511.0        3  2021-09-10T00:26:31.300   
3  2021090900      97  25511.0        4  2021-09-10T00:26:31.400   
4  2021090900      97  25511.0        5  2021-09-10T00:26:31.500   

   jerseyNumber team playDirection      x      y     s     a   dis       o  \
0          12.0   TB         right  37.77  24.22  0.29  0.30  0.03  165.16   
1          12.0   TB         right  37.78  24.22  0.23  0.11  0.02  164.33   
2          12.0   TB         right  37.78  24.24  0.16  0.10  0.01  160.24   
3          12.0   TB         right  37.73  24.25  0.15  0.24  0.06  152.13   
4          12.0   TB         right  37.69  24.26  0.25  0.18  0.04  148.33   

      dir event       o_x       o_y  
0   84.99  None  0.256121 -0.966645  
1   92.87  None  0.270096 -0.962833  
2   68.5

In [4]:
animate_play(df_tracking,df_play,df_players,df_pffScoutingData,
                2021090900,
                97,
                displayZone = True,
                displayOrientations = True).show()